requires_grad=True PyTorch 会记录它的计算图并在 .grad 中保存梯度

loss.backward() 所有可训练参数的 .grad 被填充

optimizer.step() 对应的是 param.data -= lr * param.grad 手动更新权重这句话

requires_grad=False 冻结这层参数（不求导，不更新）

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TinyNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 2)
        self.fc2 = nn.Linear(2, 2)
        self.fc3 = nn.Linear(2, 1)
    def forward(self, x):
        x = F.relu(self.fc1(x))   # layer1
        x = F.relu(self.fc2(x))   # layer2
        x = self.fc3(x)           # layer3
        return x

net = TinyNet()

# 冻结中间层 fc2
for p in net.fc2.parameters():
    p.requires_grad = False

# 输入与目标
x = torch.tensor([[1.0, 2.0]])
y_true = torch.tensor([[1.0]])

# 前向
y_pred = net(x)
loss = F.mse_loss(y_pred, y_true)
loss.backward()

# 查看梯度
for name, p in net.named_parameters():
    print(name, p.requires_grad, p.grad, p.data) # 可以发现梯度还是会向前传播的，也就是冻结的层也会参与链式法则的计算，只是自己不更新梯度


fc1.weight True tensor([[-0.0760, -0.1521],
        [-0.2768, -0.5536]]) tensor([[0.1295, 0.1109],
        [0.6487, 0.4996]])
fc1.bias True tensor([-0.0760, -0.2768]) tensor([-0.0975, -0.2738])
fc2.weight False None tensor([[0.4788, 0.3162],
        [0.1909, 0.6951]])
fc2.bias False None tensor([-0.5581,  0.3771])
fc3.weight True tensor([[ 0.0000, -3.2985]]) tensor([[0.6921, 0.1667]])
fc3.bias True tensor([-2.3890]) tensor([-0.4246])


torch.optim 是整个系统里面唯一修改参数值的模块，接收可训练参数（requires_grad=True）根据梯度 .grad 计算更新规则并修改参数 .data，也就是 param.data -= lr * param.grad 的高级封装

所有的优化器都继承自 torch.optim.Optimizer 每个优化器都有这三类成员：

params：要更新的参数（通常是 model.parameters()）

state：每个参数的历史状态（比如动量、平方梯度等）

param_groups：一组参数的配置字典（比如不同层不同学习率）

常见的优化器一共可分为两大类，分为无动量类和包含动量的类：SGD、Adagrad、Adam、AdamW、Adamax、SGD + momentum 等

In [ ]:
# 所有优化器都有这些基础参数

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=1e-3,          # 学习率
    betas=(0.9, 0.999),  # Adam 特有
    eps=1e-8,
    weight_decay=1e-4,   # 权重衰减（L2正则）
)

# 可以给不同层不同的学习率

optimizer = torch.optim.Adam([
    {'params': model.backbone.parameters(), 'lr': 1e-4},
    {'params': model.head.parameters(), 'lr': 1e-3}
])


# 完整的训练过程下，优化器一般的使用如下

for epoch in range(EPOCHS):
    optimizer.zero_grad()   # 1️⃣ 清空上次的梯度
    y_pred = model(x)       # 2️⃣ 前向传播
    loss = criterion(y_pred, y)  # 3️⃣ 计算损失
    loss.backward()         # 4️⃣ 反向传播（autograd 填充梯度）
    optimizer.step()        # 5️⃣ 参数更新

# .zero_grad() 必须放在 .backward() 前

# .step() 必须放在 .backward() 后

在 torch.optim.lr_scheduler 子模块中，可以动态修改学习率（LR Scheduling）

StepLR：每隔几步衰减 LR

MultiStepLR：在指定 epoch 衰减 LR

ExponentialLR：按指数衰减

CosineAnnealingLR：余弦退火

ReduceLROnPlateau：当验证集 loss 不再下降时自动降 LR

OneCycleLR：Transformer 常用，先升后降

get_lr()：获取当前学习率

step()：执行一次学习率更新（通常每个 epoch 调一次）

state_dict() / load_state_dict()：保存 / 恢复状态

last_epoch：当前步数或 epoch 计数

In [ ]:
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)
for epoch in range(30):
    train(...)
    scheduler.step()

weight_decay 是优化器参数 L2 正则项，防止过拟合

torch.nn.utils.clip_grad_norm_ 独立函数，限制梯度范数（防梯度爆炸）

torch.nn.utils.clip_grad_value_	独立函数，限制梯度值范围

torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

所以优化器“自动忽略冻结层”，因为它只看到那些 requires_grad=True 的参数

In [ ]:
# 下面的做法可以在训练中断的时候保存当前的模型参数和优化器

torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, 'checkpoint.pth')

checkpoint = torch.load('checkpoint.pth')
model.load_state_dict(checkpoint['model'])
optimizer.load_state_dict(checkpoint['optimizer'])

zero_grad() 清空梯度（通常每个 batch 前调用），step() 执行一次参数更新（核心）